In [141]:
import pandas as pd
 
df = pd.read_csv('better.csv')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [142]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
def remove_stop_words(text):
    text = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word.isalpha() and not word in stop_words]
    return ' '.join(text)
     
x = df.apply(lambda row: remove_stop_words(row['Pattern String']), axis=1)

In [143]:
x_train=x[0:1200]
x_test=x[1201:1400]

In [144]:
y=df["Pattern Category"]
y

0           Scarcity
1           Scarcity
2           Scarcity
3           Scarcity
4           Scarcity
            ...     
1489        Scarcity
1490        Scarcity
1491        Scarcity
1492    Misdirection
1493        Scarcity
Name: Pattern Category, Length: 1494, dtype: object

In [145]:
import keras
unique={'Social Proof': 0,
 'Misdirection': 1,
 'Urgency': 2,
 'Forced Action': 3,
 'Obstruction': 4,
 'Sneaking': 5,
 'Scarcity': 6}
# layer = keras.layers.StringLookup(vocabulary=y.unique())
# y_train=layer(y.iloc[0:1000])
# y_test=layer(y.iloc[1001:1400])
y_train=y.iloc[0:1200].apply(lambda x: unique[x])
y_test=y.iloc[1201:1400].apply(lambda x: unique[x])

In [146]:
from keras.utils import to_categorical
y_train, y_test = to_categorical(y_train, num_classes=7), to_categorical(y_test, num_classes=7)
y_train, y_test

(array([[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]], dtype=float32))

In [147]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
max_words = 20000
max_length = 500
 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(sequences, maxlen=max_length)

In [148]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Dropout, Conv1D, MaxPooling1D,GlobalMaxPooling1D
 
model = Sequential() 
model.add(Embedding(max_words, 32, input_length=max_length)) 
# model.add(Embedding(max_words, 32, input_length=max_length))
# model.add(Conv1D(128, 7, activation='relu'))
# model.add(MaxPooling1D(5))
# model.add(Conv1D(128, 7, activation='relu'))
# model.add(GlobalMaxPooling1D())
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(7, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 500, 32)           640000    
                                                                 
 flatten_17 (Flatten)        (None, 16000)             0         
                                                                 
 dense_42 (Dense)            (None, 128)               2048128   
                                                                 
 dense_43 (Dense)            (None, 7)                 903       
                                                                 
Total params: 2689031 (10.26 MB)
Trainable params: 2689031 (10.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [149]:
hist = model.fit(x_train, y_train, validation_split=0.2, epochs=20, batch_size=20)

Epoch 1/20
48/48 [==============================] - 3s 34ms/step - loss: 0.3360 - accuracy: 0.4250 - val_loss: 0.3039 - val_accuracy: 0.4750
Epoch 2/20
48/48 [==============================] - 1s 28ms/step - loss: 0.2633 - accuracy: 0.5667 - val_loss: 0.2284 - val_accuracy: 0.7167
Epoch 3/20
48/48 [==============================] - 1s 28ms/step - loss: 0.1776 - accuracy: 0.8052 - val_loss: 0.1558 - val_accuracy: 0.8792
Epoch 4/20
48/48 [==============================] - 1s 30ms/step - loss: 0.1164 - accuracy: 0.9073 - val_loss: 0.1104 - val_accuracy: 0.9042
Epoch 5/20
48/48 [==============================] - 1s 29ms/step - loss: 0.0782 - accuracy: 0.9417 - val_loss: 0.0875 - val_accuracy: 0.9292
Epoch 6/20
48/48 [==============================] - 1s 29ms/step - loss: 0.0552 - accuracy: 0.9563 - val_loss: 0.0732 - val_accuracy: 0.9292
Epoch 7/20
48/48 [==============================] - 1s 29ms/step - loss: 0.0435 - accuracy: 0.9656 - val_loss: 0.0632 - val_accuracy: 0.9500
Epoch 8/20
48

In [150]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_test)
sequences = tokenizer.texts_to_sequences(x_test)
x_test_hot = pad_sequences(sequences, maxlen=max_length)
print(x_test_hot)
score, acc = model.evaluate(x_test_hot, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

model.save("classifier-{0:.2f}.keras".format(acc))


[[  0   0   0 ...   0 151  31]
 [  0   0   0 ...   0   1   2]
 [  0   0   0 ... 153  56  16]
 ...
 [  0   0   0 ...   8 394 395]
 [  0   0   0 ...   9  23  16]
 [  0   0   0 ...   0   6   5]]
7/7 [==============================] - 0s 5ms/step - loss: 0.3981 - accuracy: 0.6231
Test score: 0.39809927344322205
Test accuracy: 0.623115599155426


In [151]:
# model.predict(x_train[1])